In [1]:
from rag_model_test import RAGRecrutamento
from preprocessing import TextPreprocessing
import settings
import pandas as pd
from google.cloud import bigquery
import datetime
import time
client = bigquery.Client.from_service_account_json(settings.GOOGLE_CREDENTIALS)

c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\willb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
full_df = client.list_rows("datathon-473001.datathon.full_data").to_dataframe()


c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [3]:
full_df.shape

(45071, 38)

In [4]:
full_df_filt = full_df[[
  'perfil_vaga_estado',
  'perfil_vaga_cidade',
  'perfil_vaga_nivel_profissional',
  'perfil_vaga_nivel_academico',
  'perfil_vaga_nivel_ingles',
  'perfil_vaga_nivel_espanhol',
  'perfil_vaga_principais_atividades',
  'perfil_vaga_competencia_tecnicas_e_comportamentais',
  'perfil_vaga_demais_observacoes',
  'infos_basicas_objetivo_profissional',
  'infos_basicas_local',
  'formacao_e_idiomas_nivel_academico',
  'formacao_e_idiomas_nivel_ingles',
  'formacao_e_idiomas_nivel_espanhol',
  'cv_pt_0',
  'situacao_candidado']]

In [5]:
# Função de tratamento
def tratar_textos(lista_textos):
    processamento = TextPreprocessing()
    textos_validos = [str(t).strip() for t in lista_textos if pd.notna(t)]
    texto = processamento.preprocess_text(texts=textos_validos,
                                          apply_lower=True,
                                          remove_ponctuation=True,
                                          remove_numbers=True,
                                          clean_html=True,
                                          apply_unidecode=True,
                                          remove_stopwords=True,
                                          remove_duplicates=True,
                                          apply_stemming=False,
                                          apply_lemmitization=False,
                                          )
    return texto


In [6]:
colunas_vaga = [
    'perfil_vaga_estado',
  'perfil_vaga_cidade',
  'perfil_vaga_nivel_profissional',
  'perfil_vaga_nivel_academico',
  'perfil_vaga_nivel_ingles',
  'perfil_vaga_nivel_espanhol',
  'perfil_vaga_principais_atividades',
  'perfil_vaga_competencia_tecnicas_e_comportamentais',
  'perfil_vaga_demais_observacoes'
]

In [7]:
colunas_candidato = [
  'infos_basicas_objetivo_profissional',
  'infos_basicas_local',
  'formacao_e_idiomas_nivel_academico',
  'formacao_e_idiomas_nivel_ingles',
  'formacao_e_idiomas_nivel_espanhol',
  'cv_pt_0'
]

In [8]:
# aplica só nas colunas escolhidas
full_df['texto_tratado_vaga'] = full_df[colunas_vaga].apply(
    lambda row: tratar_textos(row.tolist()), axis=1
)

# aplica só nas colunas escolhidas
full_df['texto_tratado_candidato'] = full_df[colunas_candidato].apply(
    lambda row: tratar_textos(row.tolist()), axis=1
)

# exemplo: pegar junto com a situacao do candidato
df_final = full_df[['situacao_candidado', 'texto_tratado_vaga', 'texto_tratado_candidato']]

In [9]:
df_final.situacao_candidado.value_counts()

situacao_candidado
prospect                          16602
encaminhado ao requisitante       13807
inscrito                           3183
não aprovado pelo cliente          2964
contratado pela decision           2254
desistiu                           2061
não aprovado pelo rh               1583
não aprovado pelo requisitante      653
entrevista técnica                  485
entrevista com cliente              363
sem interesse nesta vaga            361
em avaliação pelo rh                339
aprovado                            183
contratado como hunting             172
desistiu da contratação              48
documentação pj                       4
documentação clt                      3
documentação cooperado                2
recusado                              2
proposta aceita                       1
encaminhar proposta                   1
Name: count, dtype: int64

In [10]:
map_situacao = {
    # --- Pipeline / Em processo ---
    "prospect": "pipeline",
    "inscrito": "pipeline",
    "encaminhado ao requisitante": "pipeline",
    "entrevista técnica": "pipeline",
    "entrevista com cliente": "pipeline",
    "em avaliação pelo rh": "pipeline",
    "documentação pj": "pipeline",
    "documentação clt": "pipeline",
    "documentação cooperado": "pipeline",
    "encaminhar proposta": "pipeline",

    # --- Reprovados / Desistentes ---
    "não aprovado pelo cliente": "reprovado",
    "não aprovado pelo rh": "reprovado",
    "não aprovado pelo requisitante": "reprovado",
    "desistiu": "reprovado",
    "desistiu da contratação": "reprovado",
    "sem interesse nesta vaga": "reprovado",
    "recusado": "reprovado",

    # --- Aprovados / Contratados ---
    "aprovado": "aprovado",
    "proposta aceita": "aprovado",
    "contratado pela decision": "aprovado",
    "contratado como hunting": "aprovado"
}

In [11]:
df_final = (df_final.assign(
    situacao_candidato_grp=lambda df_: df_["situacao_candidado"].map(map_situacao)
).drop('situacao_candidado', axis=1)
)

In [12]:
df_final.situacao_candidato_grp.value_counts()

situacao_candidato_grp
pipeline     34789
reprovado     7672
aprovado      2610
Name: count, dtype: int64

In [13]:
rag_model = RAGRecrutamento()

#### Exemplo único

In [14]:
job_description = df_final.texto_tratado_vaga.iloc[355]

curriculo = df_final.texto_tratado_candidato.iloc[355]

In [15]:
resultado = rag_model.rag(curriculo, job_description)

In [16]:
curriculo

'analista rh sao paulo ensino superior completo basico dados pessoais data nascimento estado civil divorciada nacionalidade brasileira objetivo recrutamento selecao recursos humanos escolaridade universitaria faculdades guarulhos formada psicologia ano crp ativo nocoes informatica excel power point word experiencia profissional grupo souza lima ltda senior admissao atividades desenvolvidas entrevista desligamento retencao talentos analise curriculo aplicacao correcao testes psicologicos divulgacao vagas interno externo atualizacao diarias controle contas pagar setor auxilio supervisao sistema hk sarg uranet projetos sistemas tr iii work center supervisora arquivos atendimento direto telefonico cliente candidatos cadastros cobrancas dinamicas elaboracao perfil laudo emissao nota fiscal entrevistas diretivas gerencia selecionadoras orientacao recepcao treinamentos ac palografico ifp nocao departamento pessoal domestic especializada servicos domesticos psicologa selecionadora temporario c

In [17]:
job_description

'sao paulo senior ensino superior completo basico analista recrutamento selecao tech recruiter conhecimentos etapas processo triagem cv candidatos contato habilidade pacote office boa desenvoltura comunicacao vontade crescimento profissional conhecimento sobre linguagens programacao ferramentas bibliotecas boas praticas avaliacao tecnica vagas tecnologia rh necessario curso nivel agilidade contratacao profissionais vaga urgente'

In [18]:
print(resultado)

{'resumo_candidato': 'analista RH, ensino superior, psicologia, experiência em recrutamento, foco em processos, apta a análise de currículos e aplicação de testes.', 'resumo_vaga': 'Senior Recruiter - Tecnologia', 'skills_tecnicas': 'Habilidade em processo de recrutamento e seleção, conhecimento em ferramentas de gestão de candidatos, domínio de sistemas de RH (Excel, PowerPoint, Word,HK Sarg Uranet), experiência em análise de currículos, atualização de currículos, gestão de candidatos, experiência em atividades de equipe.', 'skills_softs': 'Boa desenvoltura, comunicação, vontade de crescimento profissional, capacidade de adaptação, proatividade, organização, conhecimento sobre linguagens programação, ferramentas bibliotecas, boas práticas, avaliações técnicas, conhecimento sobre processos, experiência em atividades de equipe.', 'perguntas_entrevista': ['Descreva uma situação em que você precisou adaptar sua abordagem para encontrar o candidato ideal. Qual foi o resultado?', 'Como você

In [18]:
print(type(resultado))

<class 'dict'>


#### Exemplo Geral

In [20]:
resumo_candidato = []
resumo_vaga = []
perguntas_entrevista = []
for i in range(len(df_final)):
    var_json = True
    trys = 0
    while var_json or trys < 3:
        try:
            job_description = df_final.texto_tratado_vaga.iloc[i]
            curriculo = df_final.texto_tratado_candidato.iloc[i]

            resultado = rag_model.rag(curriculo, job_description)

            # Como seu JSON tem listas de strings, vamos pegar os itens ou combinar em uma string
            candidato = resultado.get('resumo_candidato', [])
            vaga = resultado.get('resumo_vaga', [])
            perguntas = resultado.get('perguntas_entrevista', [])

            resumo_candidato.append("; ".join(candidato))
            resumo_vaga.append("; ".join(vaga))
            perguntas_entrevista.append("; ".join(perguntas))
            var_json = False
        except Exception as e:
            var_json += 1 
            print(f"Erro no índice {i}: {e}. Tentando novamente...") 

df_final = df_final.assign(
    resumo_candidato=lambda df_, rc=resumo_candidato: rc,
    resumo_vaga=lambda df_, rv=resumo_vaga: rv,
    perguntas_entrevista=lambda df_, pe=perguntas_entrevista: pe
)




Erro ao decodificar JSON. Resultado bruto:
```json
{
  "resumo_candidato": "desenvolvedor web pleno, experiência em TypeScript, JavaScript, HTML, CSS, SCSS, Go, fluentemente em inglês.  Experiência em desenvolvimento web com foco em aplicações web, possuindo certificado de inglês básico.  Busca oportunidade de trabalho em Fortaleza, com foco em PJ, oferecendo flexibilidade de horário e comunicação regular com a equipe.  Disponibilidade para início imediato.",
  "resumo_vaga": "titulo_vaga: \"Sênior Desenvolvedor Web\" ; requisitos_hards: \"TypeScript, JavaScript, HTML, CSS, SCSS, Go, Inglês básico; Certificado de inglês básico; Residência em Fortaleza; Trabalho presencial, semanal, com reuniões diárias; Flexibilidade de horário, possibilidade de prorrogamento;\" ; requisitos_softs: \"Comunicação eficaz, proatividade, organização, trabalho em equipe, resolução de problemas;\" ; extras: \"Candidato em processo, busca oportunidade de trabalho em Fortaleza, com foco em PJ, oferecendo flexi

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

X = df_final[['resumo_candidato', 'resumo_vaga']]

y = df_final['situacao_candidato_grp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
df_final.to_csv("../data/processed/df_final_rag.csv", index=False)


NameError: name 'X' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(lowercase=False) 
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear') 
start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
print('tempo decorrido: ',end_time-start_time, 'segundos')
y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import pickle

# salvar
with open("../models_project/meu_modelo.pkl", "wb") as f:
    pickle.dump(clf, f)